# Prism V2 Demo - Localhost

In [1]:
%autoawait
import time
import re
import qrcode
import uuid
import base64
import jwt
# https://pypi.org/project/termcolor/
from termcolor import colored,cprint

IPython autoawait is `on`, and set to use `asyncio`


## Import Prism V2 Controller Library

In [2]:
from prism_agent_open_api_specification_client import Client

## Instantiate Localhost Agent Controllers

In [3]:
issuer_client_controller = Client(base_url="http://host.docker.internal:8080/prism-agent")
holder_client_controller = Client(base_url="http://host.docker.internal:8090/prism-agent")
verifier_client_controller = Client(base_url="http://host.docker.internal:9000/prism-agent")

## Import Connection Management Models and API Client

In [4]:
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.types import Response

## Check Agent Connections

In [5]:
issuer_conns: Response[ConnectionCollection] = get_connections.sync(client=issuer_client_controller)
holder_conns: Response[ConnectionCollection] = get_connections.sync(client=holder_client_controller)
verifier_conns: Response[ConnectionCollection] = get_connections.sync(client=verifier_client_controller)
print("Issuer Connections:\n{}\nHolder Connections:\n{}\nVerifier Connections:\n{}".format(issuer_conns, holder_conns,verifier_conns))

Issuer Connections:
ConnectionCollection(self_='/collections', kind='Collection', contents=[], additional_properties={})
Holder Connections:
ConnectionCollection(self_='/collections', kind='Collection', contents=[], additional_properties={})
Verifier Connections:
ConnectionCollection(self_='/collections', kind='Collection', contents=[], additional_properties={})


## Issuer Connects with Holder

In [6]:
# Create Invitation
conn_request = CreateConnectionRequest()
conn_request.label = 'Connect with Holder'
issuer_invitation: Response[Connection] =  create_connection.sync(client=issuer_client_controller,json_body=conn_request)

print("Response:\n",issuer_invitation)
print("Connection ID\n",issuer_invitation.connection_id)
# print("Invitation URL\n",issuer_invitation.invitation.invitation_url)
issuer_invitationUrlbase64 = re.sub("^.*_oob=", "", issuer_invitation.invitation.invitation_url)
print("Invitation Base64\n",issuer_invitationUrlbase64)

Response:
 Connection(self_='Connection', kind='/connections/0bfa3eed-b047-47c3-a225-3ecd31a74949', connection_id='0bfa3eed-b047-47c3-a225-3ecd31a74949', state=<ConnectionAllOfState.InvitationGenerated: 'InvitationGenerated'>, created_at=datetime.datetime(2022, 12, 19, 23, 30, 50, 283081, tzinfo=tzutc()), invitation=ConnectionInvitation(id='0bfa3eed-b047-47c3-a225-3ecd31a74949', type='https://didcomm.org/out-of-band/2.0/invitation', from_='did:peer:2.Ez6LSjWAC1LgZPEjDxb1yXBby6ZYyYLJRSo45Wbpe9MZVs4R4.Vz6Mkem1XwbTctTYsfKH6Afpm1qDWwJDAzFbAUd8t3jCBTQL1.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0', invitation_url='https://domain.com/path?_oob=eyJpZCI6IjBiZmEzZWVkLWIwNDctNDdjMy1hMjI1LTNlY2QzMWE3NDk0OSIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNqV0FDMUxnWlBFakR4YjF5WEJieTZaWXlZTEpSU280NVdicGU5TVpWczRSNC5WejZNa2VtMVh3YlRjdFRZc2ZLSDZBZnBtMXFEV3dKREF6RmJBVWQ4dDNqQ

## Holder Accepts Invitation from Issuer

In [7]:
# Accept Invitation
accept_conn_request = AcceptConnectionInvitationRequest(issuer_invitationUrlbase64)
holder_invitation: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client_controller,json_body=accept_conn_request)
print(holder_invitation)

Connection(self_='Connection', kind='/connections/1c540231-4805-4a50-968f-34dc03691930', connection_id='1c540231-4805-4a50-968f-34dc03691930', state=<ConnectionAllOfState.ConnectionRequestPending: 'ConnectionRequestPending'>, created_at=datetime.datetime(2022, 12, 19, 23, 30, 51, tzinfo=tzutc()), invitation=ConnectionInvitation(id='0bfa3eed-b047-47c3-a225-3ecd31a74949', type='https://didcomm.org/out-of-band/2.0/invitation', from_='did:peer:2.Ez6LSjWAC1LgZPEjDxb1yXBby6ZYyYLJRSo45Wbpe9MZVs4R4.Vz6Mkem1XwbTctTYsfKH6Afpm1qDWwJDAzFbAUd8t3jCBTQL1.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0', invitation_url='https://domain.com/path?_oob=eyJpZCI6IjBiZmEzZWVkLWIwNDctNDdjMy1hMjI1LTNlY2QzMWE3NDk0OSIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNqV0FDMUxnWlBFakR4YjF5WEJieTZaWXlZTEpSU280NVdicGU5TVpWczRSNC5WejZNa2VtMVh3YlRjdFRZc2ZLSDZBZnBtMXFEV3dKREF6RmJBVWQ4dDNqQ0JUUUwxLl

## Check Issuer and Holder Connection State

In [8]:
# Check Connection State
issuer_conn: Response[Connection] =  get_connection.sync(client=issuer_client_controller,connection_id=issuer_invitation.connection_id)
holder_conn: Response[Connection] =  get_connection.sync(client=holder_client_controller,connection_id=holder_invitation.connection_id)

while (issuer_conn.state != 'ConnectionResponseSent' and holder_conn.state != 'ConnectionResponseReceived'):
    issuer_conn: Response[Connection] =  get_connection.sync(client=issuer_client_controller,connection_id=issuer_invitation.connection_id)
    holder_conn: Response[Connection] =  get_connection.sync(client=holder_client_controller,connection_id=holder_invitation.connection_id)
    print("Issuer State: {} Holder State: {} \n".format(issuer_conn.state,holder_conn.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestPending 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestPending 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestSent 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestSent 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestSent 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestSent 

Issuer State: ConnectionResponsePending Holder State: ConnectionRequestSent 

Issuer State: ConnectionResponsePending Holder State: ConnectionResponseReceived 

Connection established between Issuer and Holder!


## Check Agent Connections

In [9]:
issuer_conns: Response[ConnectionCollection] = get_connections.sync(client=issuer_client_controller)
holder_conns: Response[ConnectionCollection] = get_connections.sync(client=holder_client_controller)
verifier_conns: Response[ConnectionCollection] = get_connections.sync(client=verifier_client_controller)
print("Issuer Connections:\n{}\n\nHolder Connections:\n{}\n\nVerifier Connections:\n{}".format(issuer_conns, holder_conns,verifier_conns))

Issuer Connections:
ConnectionCollection(self_='/collections', kind='Collection', contents=[Connection(self_='Connection', kind='/connections/0bfa3eed-b047-47c3-a225-3ecd31a74949', connection_id='0bfa3eed-b047-47c3-a225-3ecd31a74949', state=<ConnectionAllOfState.ConnectionResponsePending: 'ConnectionResponsePending'>, created_at=datetime.datetime(2022, 12, 19, 23, 30, 50, tzinfo=tzutc()), invitation=ConnectionInvitation(id='0bfa3eed-b047-47c3-a225-3ecd31a74949', type='https://didcomm.org/out-of-band/2.0/invitation', from_='did:peer:2.Ez6LSjWAC1LgZPEjDxb1yXBby6ZYyYLJRSo45Wbpe9MZVs4R4.Vz6Mkem1XwbTctTYsfKH6Afpm1qDWwJDAzFbAUd8t3jCBTQL1.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0', invitation_url='https://domain.com/path?_oob=eyJpZCI6IjBiZmEzZWVkLWIwNDctNDdjMy1hMjI1LTNlY2QzMWE3NDk0OSIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNqV0FDMUxnWlBFakR4YjF5WEJieTZaWXlZ

## Import Credential Management Models and API Client

In [10]:
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, CreateIssueCredentialRecordRequestClaims, IssueCredentialRecordCollection
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.types import Response, Unset

## Check Credentials in Wallets|

In [11]:
issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)
print("Issuer Credentials:\n{}\n\nHolder Credentials:\n{}\n\nVerifier Credentials:\n{}".format(issuer_creds, holder_creds,verifier_creds))

Issuer Credentials:
IssueCredentialRecordCollection(items=[], offset=0, limit=0, count=0, additional_properties={})

Holder Credentials:
IssueCredentialRecordCollection(items=[], offset=0, limit=0, count=0, additional_properties={})

Verifier Credentials:
IssueCredentialRecordCollection(items=[], offset=0, limit=0, count=0, additional_properties={})


## Define Credential Schema and Content

In [13]:
firstname = input("Please enter firstname for credential: ")
lastname = input("Please enter lastname for credential: ")
birthdate = input("Please enter birthdate for credential: ")
data = {
        "firstname": firstname,
        "lastname": lastname,
        "birthdate": birthdate
      }
cred_claims = CreateIssueCredentialRecordRequestClaims()
cred_claims = cred_claims.from_dict(data)
print(cred_claims)

credential_offer = CreateIssueCredentialRecordRequest(subject_id=issuer_conn.their_did, claims=cred_claims, schema_id="1234", validity_period=3600, automatic_issuance=False, await_confirmation=False)

print(credential_offer.to_dict())

Please enter firstname for credential:  Peter
Please enter lastname for credential:  Pan
Please enter birthdate for credential:  11/11/1999


CreateIssueCredentialRecordRequestClaims(additional_properties={'firstname': 'Peter', 'lastname': 'Pan', 'birthdate': '11/11/1999'})
{'subjectId': 'did:peer:2.Ez6LSnKreYuFfaiD4RGZN4SpKbjD7ox6JTghjXzFWVzAJQtHN.Vz6Mkg2G6iwUzDFSNrFNAbB1ygjDRZTcoPYYDoBhA1KvrHsdu.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0', 'claims': {'firstname': 'Peter', 'lastname': 'Pan', 'birthdate': '11/11/1999'}, 'schemaId': '1234', 'validityPeriod': 3600, 'automaticIssuance': False, 'awaitConfirmation': False}


## Send Credential Offer to Holder

In [14]:
# Create Credential Offer
issuer_cred_offer: Response[IssueCredentialRecord] = create_credential_offer.sync(client=issuer_client_controller,json_body=credential_offer)
print(issuer_cred_offer)

IssueCredentialRecord(subject_id='did:peer:2.Ez6LSnKreYuFfaiD4RGZN4SpKbjD7ox6JTghjXzFWVzAJQtHN.Vz6Mkg2G6iwUzDFSNrFNAbB1ygjDRZTcoPYYDoBhA1KvrHsdu.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0', claims=CreateIssueCredentialRecordRequestClaims(additional_properties={'birthdate': '11/11/1999', 'firstname': 'Peter', 'lastname': 'Pan'}), record_id='f4003f93-5dc0-49c1-83fb-174e1176af26', created_at=datetime.datetime(2022, 12, 19, 23, 34, 5, 83591, tzinfo=tzutc()), role=<IssueCredentialRecordAllOfRole.ISSUER: 'Issuer'>, protocol_state=<IssueCredentialRecordAllOfProtocolState.OFFERPENDING: 'OfferPending'>, schema_id='1234', validity_period=3600.0, automatic_issuance=False, await_confirmation=False, updated_at=<prism_agent_open_api_specification_client.types.Unset object at 0x7f68d84d4a90>, publication_state=<prism_agent_open_api_specification_client.types.Unset object at 0x7f68d84d4a90>, jwt_credential=<prism_agent_open_api_spec

## Retrieve Credential Records from Issuer and Holder

In [16]:
# Check Credential State
issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)

print(colored("Current state for Issuer Credential Record {} is {} and Holder Credential Record {} is {}".format(issuer_creds.items[0].record_id,issuer_creds.items[0].protocol_state,holder_creds.items[0].record_id,holder_creds.items[0].protocol_state), "magenta", attrs=["bold"]))


Current state for Issuer Credential Record f4003f93-5dc0-49c1-83fb-174e1176af26 is OfferPending and Holder Credential Record 6fb846ee-7d03-47ef-b893-f3dec113a576 is OfferReceived


## Holder Accept Credential Offer from Issuer

In [17]:
print(holder_creds.items[0].record_id)
holder_cred_record: Response[IssueCredentialRecord] = accept_credential_offer.sync(client=holder_client_controller, record_id=holder_creds.items[0].record_id)
print(holder_cred_record)

6fb846ee-7d03-47ef-b893-f3dec113a576
IssueCredentialRecord(subject_id='did:peer:2.Ez6LSnKreYuFfaiD4RGZN4SpKbjD7ox6JTghjXzFWVzAJQtHN.Vz6Mkg2G6iwUzDFSNrFNAbB1ygjDRZTcoPYYDoBhA1KvrHsdu.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0', claims=CreateIssueCredentialRecordRequestClaims(additional_properties={'birthdate': '11/11/1999', 'firstname': 'Peter', 'lastname': 'Pan'}), record_id='6fb846ee-7d03-47ef-b893-f3dec113a576', created_at=datetime.datetime(2022, 12, 19, 23, 34, 10, tzinfo=tzutc()), role=<IssueCredentialRecordAllOfRole.HOLDER: 'Holder'>, protocol_state=<IssueCredentialRecordAllOfProtocolState.REQUESTPENDING: 'RequestPending'>, schema_id=<prism_agent_open_api_specification_client.types.Unset object at 0x7f68d84d4a90>, validity_period=<prism_agent_open_api_specification_client.types.Unset object at 0x7f68d84d4a90>, automatic_issuance=<prism_agent_open_api_specification_client.types.Unset object at 0x7f68d84d4a90>, aw

## Issue Credential to Holder

In [21]:
print(issuer_creds.items[0].record_id)
issuer_cred_record: Response[IssueCredentialRecord] = issue_credential.sync(client=issuer_client_controller, record_id=issuer_creds.items[0].record_id)
print(issuer_cred_record)

f4003f93-5dc0-49c1-83fb-174e1176af26
IssueCredentialRecord(subject_id='did:peer:2.Ez6LSnKreYuFfaiD4RGZN4SpKbjD7ox6JTghjXzFWVzAJQtHN.Vz6Mkg2G6iwUzDFSNrFNAbB1ygjDRZTcoPYYDoBhA1KvrHsdu.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0', claims=CreateIssueCredentialRecordRequestClaims(additional_properties={'birthdate': '11/11/1999', 'firstname': 'Peter', 'lastname': 'Pan'}), record_id='f4003f93-5dc0-49c1-83fb-174e1176af26', created_at=datetime.datetime(2022, 12, 19, 23, 34, 5, tzinfo=tzutc()), role=<IssueCredentialRecordAllOfRole.ISSUER: 'Issuer'>, protocol_state=<IssueCredentialRecordAllOfProtocolState.CREDENTIALPENDING: 'CredentialPending'>, schema_id='1234', validity_period=3600.0, automatic_issuance=False, await_confirmation=False, updated_at=datetime.datetime(2022, 12, 19, 23, 34, 23, tzinfo=tzutc()), publication_state=<prism_agent_open_api_specification_client.types.Unset object at 0x7f68d84d4a90>, jwt_credential=<prism_

## Check if Credential is Successfully Issued to Holder

In [22]:
# Check Credential State
issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
verifier_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=verifier_client_controller)

print(colored("Current state for Issuer Credential Record {} is {} and Holder Credential Record {} is {}".format(issuer_creds.items[0].record_id,issuer_creds.items[0].protocol_state,holder_creds.items[0].record_id,holder_creds.items[0].protocol_state), "magenta", attrs=["bold"]))

while (issuer_creds.items[0].protocol_state != 'CredentialSent' and holder_creds.items[0].protocol_state != 'CredentialReceived'):
    issuer_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=issuer_client_controller)
    holder_creds: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=holder_client_controller)
    print("Issuer State: {} Holder State: {} \n".format(issuer_creds.items[0].protocol_state,holder_creds.items[0].protocol_state))
    time.sleep(1)
    
print(colored("Credential Record {} Issued to Holder! Continue with notebook".format(issuer_creds.items[0].record_id), "green", attrs=["bold"]))


Current state for Issuer Credential Record f4003f93-5dc0-49c1-83fb-174e1176af26 is CredentialPending and Holder Credential Record 6fb846ee-7d03-47ef-b893-f3dec113a576 is RequestSent
Issuer State: CredentialPending Holder State: RequestSent 

Issuer State: CredentialPending Holder State: RequestSent 

Issuer State: CredentialPending Holder State: RequestSent 

Issuer State: CredentialPending Holder State: RequestSent 

Issuer State: CredentialPending Holder State: RequestSent 

Issuer State: CredentialPending Holder State: RequestSent 

Issuer State: CredentialGenerated Holder State: RequestSent 

Issuer State: CredentialGenerated Holder State: RequestSent 

Issuer State: CredentialGenerated Holder State: RequestSent 

Issuer State: CredentialGenerated Holder State: RequestSent 

Issuer State: CredentialGenerated Holder State: RequestSent 

Issuer State: CredentialGenerated Holder State: RequestSent 

Issuer State: CredentialGenerated Holder State: RequestSent 

Issuer State: Credential